In [ ]:
!pip install scikeras scikit-learn-intelex threadpoolctl -q

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
# import os
# import re
# from __future__ import print_function
from datetime import datetime

# import string
# import spacy
# import nltk
# from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold,GridSearchCV

import tensorflow as tf   
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OrdinalEncoder

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.python.keras import Sequential
from keras.layers import Embedding, LSTM, Dense,Dropout
from scikeras.wrappers import  KerasClassifier
from sklearn.utils.class_weight import compute_class_weight
# from sklearn.pipeline import Pipeline
# from keras.utils import to_categorical

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV 
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix, f1_score,precision_score, recall_score

# Load data

In [ ]:
df = pd.read_csv('../data/processed/drug_review_clean.csv', index_col= False)

# Preprocess

# Build a class to do preprocess
The dataframe contains different types of features: numericals ('mean_word_len','word_count', etc), categorical(eg.'rating_category','condidition','drugName'), and datetime ('date'). Also, The target of 'sentiment_label' is categorimcal. The preprocess including the following steps:

1.tokenizer the'review_clean' using keras Tokenizer
2. encode the categorical features and target 'sentiment_label'
3. extract the 'date' to several new features 'year','month','day'.
4.scale the numerical features using MinMaxScaler.
5. train test split

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, max_sequence_length=214):
        self.max_sequence_length = max_sequence_length
        self.tokenizer = None
    """ define a function to convert text to tokenizer """
    def fit(self, X, y=None):
        self.tokenizer = Tokenizer(num_words = 5000, lower = False)
        self.tokenizer.fit_on_texts(X)
        return self
    """ define a function to convert the review text into sequence """

    def transform(self, X):
        sequences = self.tokenizer.texts_to_sequences(X)
        return pad_sequences(sequences, maxlen= 200)


In [ ]:
class NumericalScaler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.scaler = MinMaxScaler()
        self.scaler.fit(X)
        return self

    def transform(self, X):
        return self.scaler.transform(X)

In [ ]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ordinal_encoder = OrdinalEncoder()
        
    def fit(self, X, y = None):
        
        self.ordinal_encoder.fit(X)
        return self

    def transform(self,X):
        return self.ordinal_encoder.transform(X)

In [ ]:
class DateExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self,X):
        X['date'] = pd.to_datetime(X['date'])
        X['year'] = X['date'].dt.year
        X['month'] = X['date'].dt.month
        X['day'] = X['date'].dt.day
        return X[['year','month','day']]             

In [ ]:
X = df[
    [
        "drugName",
        "condition",
        "rating",
        "date",
        "usefulCount",
        "rating_category",
        "review_clean",
        "review_len",
        "mean_sentence_len",
        "word_count",
        "mean_word_len",
        "unique_word_count",
        "sentiment_subjectivity",
        "sentiment_score",
        "genuine_positive",
        "genuine_negative",
        "genuine_neutral",
    ]
]
y = df[["sentiment_label"]]

In [ ]:
# Transfrom each feature
df["review_clean"] = df["review_clean"].apply(lambda x: " ".join(x.split()[:200]))
X_text = TextPreprocessor(max_sequence_length=214).fit_transform(df["review_clean"])
numerical_cols = [
    "rating",
    "usefulCount",
    "review_len",
    "mean_sentence_len",
    "word_count",
    "mean_word_len",
    "unique_word_count",
    "sentiment_subjectivity",
    "sentiment_score",
]
X_numerical = NumericalScaler().fit_transform(df[numerical_cols])
X_drugName = CategoricalEncoder().fit_transform(df[["drugName"]])
X_condition = CategoricalEncoder().fit_transform(df[["condition"]])
X_date = DateExtractor().fit_transform(df)

In [ ]:
#combine the features
X_transformed = np.concatenate([X_text, X_numerical, X_drugName,
                                X_condition, X_date], axis=1)

In [ ]:
#encode targe feature
sentiment_label_encode = LabelEncoder()
y_encode = sentiment_label_encode.fit_transform(df['sentiment_label'])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y_encode, test_size=0.25, random_state=123
)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
unique_labels= np.unique(y_train)
class_weights = compute_class_weight(class_weight="balanced", classes=unique_labels, y=y_train)
class_weight_dict = dict(zip(unique_labels, class_weights))
for label, weight in class_weight_dict.items():
    print(f'Class:{label}, weight:{weight:.2f}')

There weight for postive sentiment (class1) is 3.15, for negtive is 0.53(class 2) and for neutral is 1.26(class 0). This imbalance will be addressed during modeling.

# Modeling


The objective of the model is to predict sentiment accurately while mitigating bias. The datased isimbalanced with more positve sentiments. Employing accuracy as the primary metric may lead to misleading conclusiions. To avoid this.  The F1 score will serve as the principle metric. Meanwhile, metrics such as traning time, preciison, recall and accuracy scale for each model will be recored. The chosen model will ideally strikes a balance between predictive performance, computational efficiency and the ability to capture sentiment in drug review.

Three models will be exlpored in this sentiment analysis task:

1. Multinomial Naive Bayes(MNB)
   it is a probalilistic classification algorithm based on Bayes therem and it is widely used in processing text data.  it assumes independence between features. the simlicity and efficiency make it a great baseline model for text based sentiment analysis taks.

2. Long short term memory(LSTM)
  This is a type of recurrent neural network RNN designed for sequential data propressing. For sentiment analysis, LSTM excels   in capturing dependencies and nuances in text information over extended sequences. Its abaility to retain and forget information  makes it well suited for tasks  where context plays a crucial role, e.g undersing sentiment in long text.
3. Random Forest (RF)
   It is an effective ensemble learning method in classification. It can provide feature importances, which can help extend the understanding  of model process.

## Model Metrics Comparision

build a method to evaluate model performance

In [ ]:
metrics_list =[]
pd.DataFrame(columns = ['model', 'accuracy', 'precision','recall','F1 score'])

def model_metrics(model, y_true, y_pred, best_params = None):
    f1 = f1_score(y_true, y_pred, average="macro")
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    print(f"\n model performance:")
    if best_params:
        print(f"Best hyperparameters: {best_params}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")


    
    metrics_list.append({'model': model,
                                'accuracy':accuracy,
                                'precision': precision,
                                'recall': recall,
                                'F1-score':f1,
                                })

    print(f'\n {model} Classification Report:')
    print(classification_report(y_true, y_pred))
   
    """plot comfusion matrix"""
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(15, 6))
    sns.heatmap(
        cm,
        annot=True,
        cmap="Blues",
        xticklabels=["class 0 neutral", "class 1 positive", "class 2 negative"],
        yticklabels=["class 0", "class 1", "class 2"],
    )
    plt.xlabel("predicted Labels")
    plt.ylabel("actual labels")
    plt.title(f"confusion matrix ")
    plt.show()

    

## Multinomial Naive Bayes Model (MNB)

In [ ]:
mnb_model = MultinomialNB()
mnb_param_grid = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0, ],
          'fit_prior': [True, False]
         }
mnb_grid = RandomizedSearchCV(mnb_model, param_distributions=mnb_param_grid, n_jobs=-1, cv=5, verbose=1)

print("MNB Start Time", datetime.now().strftime("%H:%M:%S"))
mnb_grid.fit(X_train,y_train)
print("MNB End Time =", datetime.now().strftime("%H:%M:%S"))


MNB shows a fast training time.

In [ ]:
best_mnb_params = mnb_grid.best_params_
y_mnb_pred = mnb_grid.best_estimator_.predict(X_test)

In [ ]:
model_metrics(mnb_grid.best_estimator_, y_test, y_mnb_pred, best_mnb_params)

## Random Forest Model

In [ ]:
rf =  RandomForestClassifier()
random_grid = {
               'max_depth': [5, 10],
               'max_features': ['log2','sqrt', None],
               'min_samples_leaf': [4, 8],
               'n_estimators': [25, 50]}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)

from datetime import datetime
print("Start Time =", datetime.now().strftime("%H:%M:%S"))
rf_random_result = rf_random.fit(
    X_train, y_train
)
print("End Time =", datetime.now().strftime("%H:%M:%S"))


best_rf_params = rf_random.best_params_
best_rf_model = rf_random.best_estimator_

y_rf_pred = best_rf_model.predict(X_test).round() 
model_metrics(best_rf_model, y_test, y_rf_pred, best_rf_params)

## LSTM model

In [ ]:
# build a LSTM sequential model
def create_lstm_model(dropout_rate=0, epochs=5, batch_size=32):
    lstm_model = tf.keras.models.Sequential()
    lstm_model.add(Embedding(input_dim=5000, output_dim=32, input_length=214))
    lstm_model.add(LSTM(100))
    lstm_model.add(Dropout(0.2))
    lstm_model.add(Dense(3, activation="softmax"))
    lstm_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    lstm_model.summary()
    return lstm_model

In [ ]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=123)
lstm_model = KerasClassifier(
    model=create_lstm_model, dropout_rate=None, epochs=1, batch_size=None, verbose=1
)

In [ ]:
#hyperameter tunning
param_grids = {
    'dropout_rate': [0.0, 0.2],
    'epochs': [5,10],
    'batch_size': [32, 64],
    'class_weight': [class_weight_dict]
}

In [ ]:
from sklearn.metrics import make_scorer
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=123)
#model metrics
scorers = {
            'f1_score': make_scorer(f1_score, average='macro'),
            'precision_score': make_scorer(precision_score, average='macro'),
            'recall_score': make_scorer(recall_score, average='macro'),
            'accuracy_score': make_scorer(accuracy_score)
          }

lstm_grid = RandomizedSearchCV(
    estimator=lstm_model,
    param_distributions=param_grids,
    cv=kfold,
    scoring = scorers,
    verbose=1, error_score='raise',n_jobs=-1, refit='f1_score'
)

In [ ]:
from datetime import datetime
print("Start Time =", datetime.now().strftime("%H:%M:%S"))
lstm_grid_result = lstm_grid.fit(
    X_train, y_train, batch_size=None,
    epochs=3,
    verbose=1,
   
)
print("End Time =", datetime.now().strftime("%H:%M:%S"))

In [ ]:
best_lstm_params = lstm_grid.best_params_
best_lstm_model = lstm_grid.best_estimator_
y_lstm_pred = best_lstm_model.predict(X_test)
model_metrics(best_lstm_model, y_test, y_lstm_pred, best_lstm_params)

# Save The Best Model

In [ ]:
import joblib

The RandomForest model is the top performing model with F1 score of 0.99 and a fast training time.

In [ ]:
# save the model
joblib.dump(best_rf_model, '../Desktop/Capstone_Three_Project/models/best_rf_model.joblib')

load_model = joblib.load('best_rf_model.joblib')

# Summary